
<img src="../resources/cropped-SummerWorkshop_Header.png">  

<h1 align="center">Visual Behavior Ophys</h1> 
<h2 align="center">Summer Workshop on the Dynamic Brain </h2> 
<h3 align="center">August, 2021</h3> 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>Here we build on the classification tutorial to apply these same concepts to real neural data

</div>


# Apply classification (decoding) to neural data
The previous classification tutorial applied classification on low-dimensional (2D or 3D) datasets. We will now extend this concept to actual (higher dimensional) neural data from the Visual Behavior dataset. 

In the example below, we will attempt to classify the neural response based on the stimulus that elicited that response. This is a form of 'decoding'. In other words, we will attempt to decode the stimulus based only on the neural response.

In the following example, the number of feature dimensions will be much higher. We will use the average response of each simultaneously recorded neuron in a short window following the stimulus as our feature matrix. Thus, our number of features will be equal to the number of simultaneously recorded neurons.

It's important to note that the choice of which feature to use is a design choice on the part of the scientist. Instead of using the average response, we could make many other choices such as using the max response, the instantaneous response at some point in time, or even maintaining the full dynamic timecourse (though this latter choice would dramatically increase our dimensionality).

## first some imports

In [ ]:
# os is a library of standard operating system functions
import os

# numpy is a library of mathematical functions for manipulating arrays
import numpy as np

# pandas is a library of data analysis tools
import pandas as pd

# matplotlib is a plotting library
import matplotlib.pyplot as plt

# seaborn is a library of plotting functions built on top of matplotlib
import seaborn as sns

# The allensdk is the institute toolset of interacting with data
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

# mindscope_utilities contains convenience functions built on top of the AllenSDK
import mindscope_utilities
import mindscope_utilities.visual_behavior_ophys as ophys

# This sets the number of columns that will be displayed by default when viewing Pandas tables
pd.set_option('display.max_columns', 500)

In [ ]:
import platform
platstring = platform.platform()

if ('amzn1' in platstring):
    # for AWS
    data_storage_directory = "/data/visual-behavior-ophys-data"
    # use local cache for AWS
    cache = VisualBehaviorOphysProjectCache.from_local_cache(cache_dir=data_storage_directory, use_static_cache=True)
else:  
    # for local drive, different operating systems
    if 'Darwin' in platstring:
        # OS X 
        data_root = "/Volumes/Brain2021/"
    elif 'Windows'  in platstring:
        # Windows (replace with the drive letter of USB drive)
        data_root = "E:/"
        #data_root = r'//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/platform_paper_cache'
    else:
        # your own linux platform
        # EDIT location where you mounted hard drive
        data_root = "/media/$USERNAME/Brain2021/"
        data_root = "/run/media/tom.chartrand/Brain2021"
        
    # visual behavior cache directory
    cache_dir = manifest_path = os.path.join(data_root, "visual_behavior")
    # use from_s3_cache for loading from local directory
    cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=cache_dir)

## Next we need to get some data loaded

### Set up data paths, load session/experiment table
Recall from the Visual Behavior data overview that we have hundreds of behavior/ophys sessions, each with up to 8 simultaneously acquired imaging planes (aka 'experiments')

In [ ]:
cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=data_root)

In [ ]:
session_table = cache.get_ophys_session_table()
experiment_table = cache.get_ophys_experiment_table()

### Load one example session
We are going to select one session from the session table, session 990139534. This is a session with Sst-IRES-Cre mouse, which expressed GCaMP6f in somatostatin positive inhibitory interneurons. There were 6 simultaneously acquired imaging planes for this session. 
We can view metadata for this session as follows:

In [ ]:
ophys_session_id = 854060305
session_table.loc[ophys_session_id]

In [ ]:
# Note the difference in selection:
print(type(session_table.loc[ophys_session_id])) # single brackets 
print(type(session_table.loc[[ophys_session_id]])) # double brackets

### Download all associated experiments

Each session consists of one or more 'experiments', in which each experiment is a single imaging plane

Each mesoscope session has up to 8 experiments (this session has 6) associated with the session. We will load all sessions into a dictionary with the experiment IDs as the keys

The first time that this cell is run, the associated NWB files will be downloaded to your local `data_storage_directory`. Subsequent runs of this cell will be faster since the data will already be cached locally.

In [ ]:
experiments = {}
ophys_experiment_ids = session_table.loc[ophys_session_id]['ophys_experiment_id']
for ophys_experiment_id in ophys_experiment_ids:
    experiments[ophys_experiment_id] = cache.get_behavior_ophys_experiment(ophys_experiment_id)

## Restructure neural data

The cell below will load the neural data into a pandas 'tidy' format by iterating over each of the 6 experiments and using some helpful tools from the `visual_behavior_ophys` module of the `mindscope_utilities` package that was imported above as `ophys`. 

It will also include a subset of metadata from `ophys_experiment_table` to facilitate splitting by depth, structure (aka cortical area), cre line (aka cell class), etc.

Note that 'tidy' data means that each row represents only one observation. Observations are stacked vertically. Thus, the `timestamps` colums will repeat for every cell in the dataset.

Also note that this could fail if you are on a machine or cloud instance with too little available RAM.

In [ ]:
# you can also use tqdm package to display progress bar.
# EXAMPLE:
# _________
# from tqdm import tqdm
# for i in tqdm(I):


In [ ]:
# create an empty list
neural_data = []
for ii, ophys_experiment_id in enumerate(experiments.keys()): 
    print('on experiment {} of {}'.format(ii+1, len(experiments)), end='\r')
    this_experiment = experiments[ophys_experiment_id]
    
    # append the data for this experiment to a list
    neural_data.append(ophys.build_tidy_cell_df(this_experiment))
    
# concatate the list of dataframes into a single dataframe
neural_data = pd.concat(neural_data)

In [ ]:
neural_data.head()

In [ ]:
len(neural_data)

This dataframe is over 2 million rows long

In [ ]:
len(neural_data['cell_specimen_id'].unique())

And it contains the timeseries of 53 unique simultaneously recorded neurons

## Set up data for a classification algorithm and scikit-learn
We need to get the data into a standard format for the classification algorithms in scikit learn, which is often a feature matrix (**`X`**) and a vector of labels (**`y`**).

For this analysis, let's look at the responses to each of the **8 unique visual stimuli** in this session, plus the response to **omitted** stimuli.

### make some decisions about the format of `X`
Our goal is to construct a feature matrix that consists of the the average response of each cell in a short (750ms) window following each stimulus on every stimulus presentation. Each stimulus_presentation will be represented by an `n_neurons` dimension feature vector. Our feature matrix, `X`, will thus be `n_trials x n_neurons`

But as noted in the introduction, we could have made many other choices about the format of `X`. An interesting exercise would be to repeat the analyses that follow with different choices for the feature matrix `X`.

### Calculate the full response to each stimulus
First, we will calculate an event triggered response to each stimulus start time in the stimulus table.

To start with, let's define a simple wrapper function on the `mindscope_utilities.event_triggered_response` function. We are going to use the deltaF/F response in a window from 0 to 750 ms from every stimulus event to calculate the response:

In [ ]:
def calculate_event_triggered_response(single_cell_data, event_times):
    event_triggered_response = mindscope_utilities.event_triggered_response(
        single_cell_data,
        t = 'timestamps',
        y = 'dff',
        event_times = event_times,
        t_before = 0,
        t_after = 0.75,
        output_sampling_rate = 30 
    )
    
    return event_triggered_response

We are going to define our 'event_times' as the start time of *every* unique stimulus. First we will load the stimulus table from one of our experiments. Recall that each experiment in a given session shares the same stimulus data:

In [ ]:
stimulus_presentations = experiments[ophys_experiment_ids[0]].stimulus_presentations.drop(columns = ['image_set']) # dropping the 'image_set' column to avoid confusion. Image_set column contains a unique string for set of images presented in a session.
stimulus_presentations.head(10)

Then we will select our event times as the `start_time` of each stimulus

In [ ]:
event_times = stimulus_presentations['start_time']

Now iterate over every cell and apply the above function to build out an event triggered response for every cell/stimulus

In [ ]:
# instantiate an empty list. We will append every event triggered response to this list
full_etr = []

# iterate over each unique cell
cell_specimen_ids = neural_data['cell_specimen_id'].unique()

for cell_count, cell_specimen_id in enumerate(cell_specimen_ids):
    print('on cell {}, #{} of {}'.format(cell_specimen_id, cell_count+1, len(cell_specimen_ids)), end='\r')
    
    # calculate the event triggered response for this cell
    full_etr_this_cell = calculate_event_triggered_response(
        neural_data.query('cell_specimen_id == @cell_specimen_id'),
        event_times
    )
    
    # add a column identifying the cell_specimen_id
    full_etr_this_cell['cell_specimen_id'] = cell_specimen_id
    # append to our list
    full_etr.append(full_etr_this_cell)
    
# concatenate our list of dataframes into a single dataframe
full_etr = pd.concat(full_etr)

The resulting dataframe includes the full response of every cell to every stimulus event on a common 30 Hz timebase. We can view it as follows.

In [ ]:
full_etr

But as noted above, what we're interested in is just the *average* response of each cell to each stimulus. We can use the Pandas `groupby` command to group by `cell_specimen_id` and `event_number`

In [ ]:
average_responses = full_etr.groupby(
    ['cell_specimen_id','event_number']
)[['dff']].mean().reset_index()
average_responses

The `event_triggered_response` function returns a column called `event_number`. Because we passed in every event in the stimulus table, this number will correspond directly with the `stimulus_presentations_id` column in the stimulus table (see above). We will rename it for clarity and to facilitate merges below:

In [ ]:
average_responses.rename(columns={'event_number':'stimulus_presentations_id'}, inplace=True)
average_responses

Then we can easily merge in stimulus information from the stimulus table by merging on the `stimulus_presentations_id` column:

In [ ]:
average_responses = average_responses.merge(
    stimulus_presentations,
    on = 'stimulus_presentations_id',
    how = 'left'
)
average_responses

Now we can construct a dataframe called `features_and_labels` that will contain one row per trial, one column per cell, plus columns with the image_index and image_name

In [ ]:
features_and_labels = average_responses.pivot(
    index = 'stimulus_presentations_id',
    columns = 'cell_specimen_id',
    values = 'dff'
).merge(
    stimulus_presentations[['image_index','image_name']],
    on = 'stimulus_presentations_id',
    how = 'left'
)
features_and_labels.sample(10)

The `X` matrix can be extracted by getting the columns associated with the cell_specimen_ids

In [ ]:
X = features_and_labels[cell_specimen_ids]
X.sample(10)

And `y` is just the image_name column (it could also be the image_index column if you want a numeric value instead of a string to represent the image identity)

In [ ]:
y = features_and_labels['image_name']
y.sample(10)

## Dimensionality reduction
One way of visualizing the data is to use a dimensionality reduction technique. For example, we can use t-SNE, which will project our n-neuron-dimensional feature space into two dimensions.

In [ ]:
from sklearn.manifold import TSNE

X_embedded = TSNE(n_components=2).fit_transform(X.values)

In [ ]:
 X_embedded.shape

And visualize the results, with colors representing each unique stimulus.

In [ ]:
features_and_labels['tsne-2d-one'] = X_embedded[:,0]
features_and_labels['tsne-2d-two'] = X_embedded[:,1]
plt.figure(figsize=(16,10))
ax = sns.scatterplot(
    data=features_and_labels,
    x="tsne-2d-one", 
    y="tsne-2d-two",
    hue="image_name",
    hue_order = np.sort(features_and_labels['image_name'].unique()),
    palette='tab10',
    legend="full",
    alpha=0.3
)

This demonstrates that the time-averaged population responses to at least some of the stimuli seem to fall into distinct clusters in our higher dimensional space, while others appear more overlapped. This implies that a decoding analysis might be more successful at decoding some stimuli than others.

## Train Linear Discriminant Analysis decoder
We can use an Linear Discriminant Analysis decoder from scikit learn to ask how well we can decode image identity from the feature matrix we have constructed. 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix, accuracy_score

Split our data into train and test sets, instantiate the model, then fit.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
model = LDA(store_covariance=True)
model.fit(X_train, y_train);

Use the model to make predictions on the held-out test set

In [ ]:
y_pred = model.predict(X_test)
y_pred

Evaluate the accuracy

In [ ]:
accuracy_score(y_test, y_pred)

We can construct a confusion matrix as follows. Note that we are converting the matrix to a dataframe just to make axis labeling easier.

In [ ]:
confusion_df = pd.DataFrame(
    confusion_matrix(y_test, y_pred, normalize = True), 
    columns = ['predicted_{}'.format(im) for im in model.classes_],
    index = ['actual_{}'.format(im) for im in model.classes_]
)
confusion_df

In [ ]:
# if above normalization step doesnt work:
# confusion_df = confusion_df/confusion_df.sum()

And we can also visualize our confusion matrix:

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
im = ax.imshow(confusion_df, aspect='auto')

ax.set_xticks(np.arange(0,9))
ax.set_xticklabels(model.classes_)
ax.set_xlabel('predicted stimulus')

ax.set_yticks(np.arange(0,9))
ax.set_yticklabels(model.classes_)
ax.set_ylabel('actual stimulus')

plt.colorbar(im, label='fraction correct')

## we can plot the feature matrix after sorting by image ID
This helps explain how the classifier might be working: some cells are very image selective.

In [ ]:
fig, ax = plt.subplots(figsize = (15,5))

sorted_matrix = features_and_labels.sort_values(by='image_index')
ax.imshow(sorted_matrix[cell_specimen_ids], aspect='auto', clim=[0,0.1], interpolation='none')

# add a white line to demarcate every distinct image
demarcations = []
sorted_matrix['last_image_index'] = sorted_matrix['image_index'].shift()
for idx, row in sorted_matrix.reset_index().iterrows():
    if row['last_image_index'] != row['image_index']:
        ax.axhline(idx, color='white')
        demarcations.append(idx)
demarcations.append(idx)

# set yticks at the halfway points between demarcations
ax.set_yticks([(demarcations[i] + demarcations[i+1])/2 for i in range(len(demarcations)-1)])      
# assign yticklabels as image_name
ax.set_yticklabels(sorted_matrix['image_name'].unique())

ax.set_ylabel('trials (sorted by image_name)')
ax.set_xlabel('neuron')

for i in range(0, len(cell_specimen_ids), 1):
    ax.axvline(i-0.5, color='white')


ax.set_title('average responses sorted by image index')

## And here are the images that the mouse saw.

In [ ]:
experiment = experiments[ophys_experiment_ids[0]]
fig, ax = plt.subplots(2,4,figsize = (20,8), sharex = True, sharey=True)
for ii,image_name in enumerate(experiment.stimulus_templates.index):
    ax.flatten()[ii].imshow(experiment.stimulus_templates.loc[image_name]['unwarped'], cmap='gray')
    ax.flatten()[ii].set_title(image_name)
fig.tight_layout()